# Chains

In [2]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

In [3]:
import os 
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [4]:
llm = OpenAI(temperature=.9)
prompt  = ChatPromptTemplate.from_template("Who won the Wimbledon for year {year}?")
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
chain.run('2021')

'\n\nComputer: Novak Djokovic won the 2021 Wimbledon.'

## Simple Sequential Chain
The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [9]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [10]:
llm = OpenAI(temperature=.1)
template = """Who is the president of the {country} ?"""
prompt_template = PromptTemplate(input_variables=["country"], template=template)
president_chain = LLMChain(llm=llm, prompt=prompt_template)

In [11]:
template = """What is the birth date of {president}?"""
prompt_template = PromptTemplate(input_variables=["president"], template=template)
bdate_chain = LLMChain(llm=llm, prompt=prompt_template)

In [12]:
overall_chain = SimpleSequentialChain(chains=[president_chain, bdate_chain], verbose=True)

In [13]:
review = overall_chain.run("India")



> Entering new SimpleSequentialChain chain...


The current President of India is Ram Nath Kovind.


Ram Nath Kovind was born on October 1, 1945.

> Finished chain.


## SequentialChain 
A more general form of sequential chains, allowing for multiple inputs/outputs.

In [14]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

llm = OpenAI(temperature=.1)

In [15]:
template = """Could you add few lines about {topic} and {subtopic}?"""
prompt_template = PromptTemplate(input_variables=["topic", "subtopic"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [16]:
template = """Summarise the following synopsis to one line: {synopsis} """
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
summary_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="oneLine")

In [17]:
template = """Translate the following synopsis to spanish:  {synopsis}"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
translated_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="translatedsynopsis")

In [18]:
overall_chain = SequentialChain(
    chains=[synopsis_chain,translated_chain, summary_chain],
    input_variables=["topic", "subtopic"],
    output_variables=["synopsis", "oneLine", "translatedsynopsis"],
    verbose=True)

In [19]:
overall_chain({"topic":"Spain", "subtopic": "Economy"})



> Entering new SequentialChain chain...

> Finished chain.


{'topic': 'Spain',
 'subtopic': 'Economy',
 'synopsis': '\n\nSpain is a major European economy and a member of the European Union. It is the 14th largest economy in the world and the fifth largest in the European Union. The Spanish economy is highly diversified, with sectors such as tourism, manufacturing, energy, construction, and finance all contributing to its growth. Spain has a strong agricultural sector, with the production of olives, wine, and citrus fruits being particularly important. The country is also a major producer of renewable energy, with wind and solar energy being the main sources. In recent years, Spain has seen a surge in foreign investment, particularly in the technology sector, which has helped to boost the economy.',
 'oneLine': '\n\nSpain is a major European economy and a member of the European Union, with a highly diversified economy and strong agricultural sector, as well as a surge in foreign investment in the technology sector.',
 'translatedsynopsis': '\n\

## LoadSummarize chain

This is the fundamental chain to perform the task of summarization over documents.

### Map reduce
The map reduce documents chain first applies an LLM chain to each document individually, treating the chain output as a new document. It then passes all the new documents to a separate combine documents chain to get a single output (the Reduce step). It can optionally first compress, or collapse, the mapped documents to make sure that they fit in the combine documents chain (which will often pass them to an LLM). This compression step is performed recursively if necessary.

This algorithm works by first splitting the entire input into small chunks using a text splitter. Then we create a summary of each of these chunks. Once we get a summary of each, the algorithm creates a summary over these summaries and provides an output

In [96]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

In [99]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [102]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

In [105]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)
chain({"input_documents": docs}, return_only_outputs=True)

In [111]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run( docs)

' President Biden has outlined a Unity Agenda for the Nation, which includes tackling the opioid epidemic, mental health issues, and supporting veterans. In response to Russian aggression in Ukraine, the US and its allies are taking action to seize ill-gotten gains and isolate them economically. The US is also providing military, economic, and humanitarian assistance to Ukraine. The American Rescue Plan and the Bipartisan Infrastructure Law have been passed to help Americans struggling with the pandemic and to create jobs. The President is also pushing for the passage of the Bipartisan Innovation Act, which would make record investments in emerging technologies and American manufacturing. He is encouraging the use of American products to support American jobs and is proposing a global minimum tax rate to prevent companies from avoiding taxes. Finally, he is calling for Americans to come together and move forward on COVID-19 and other issues.'

# Router
Router chain dynamically selects the next chain to use for a given input.

In [112]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [113]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [114]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [115]:
llm = OpenAI()
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [116]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [117]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [118]:
chain = MultiPromptChain(router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,)

In [120]:
print(chain.run("What is Newtons third law?"))



> Entering new MultiPromptChain chain...
physics: {'input': "What is Newton's third law?"}
> Finished chain.


Newton's third law states that for every action, there is an equal and opposite reaction. In other words, the forces between two objects always come in pairs, and these forces are equal in magnitude and opposite in direction.


In [121]:
print(chain.run("What is the largest 3 digit prime number?"))



> Entering new MultiPromptChain chain...
math: {'input': 'What is the largest three digit prime number?'}
> Finished chain.


The largest three digit prime number is 997.


In [20]:
print(chain.run("Where did the First World War take place?"))



Computer: In 2019, Novak Djokovic won the Wimbledon Men's Singles title. The First World War took place primarily in Europe, with the main battles taking place in France, Belgium, and Germany.
